In [5]:
def clearvars():    
    for el in sorted(globals()):
        if '__' not in el:
                del el
clearvars()

In [6]:
import sys
import sklearn
import numpy as np
import os
import pandas as pd

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)
import seaborn as sn

In [7]:
csv_file = "C:/Users/rodri/Desktop/NCFS2/ML/project/data/SU_ORNEKLEM_KK_HAR_BILGI.txt"
df = pd.read_csv(csv_file,sep=',',encoding='latin-1')

C:\Users\rodri\anaconda3\envs\AppliedMachineLearning\lib\site-packages\IPython\core\interactiveshell.py:3457: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [8]:
df.head()

,MUSTERI_ID_MASK,ISLEM_TARIHI,ISLEM_SAAT,ISLEM_TUTARI,MCC,UYEISYERI_ID_MASK,ONLINE_ISLEM,HARCAMA_TIPI,DOVIZ_FMT,X,Y
0,11861396,23OCT2014:00:00:00,21:01:13,10.57,RESTORAN,999999,0,AV,TURK_LIRASI,NaN,NaN
1,13667756,12APR2015:00:00:00,15:18:50,82.00,RESTORAN,999999,0,AV,TURK_LIRASI,NaN,NaN
2,11024865,23JUL2014:00:00:00,21:12:12,15.00,RESTORAN,999999,0,AV,TURK_LIRASI,NaN,NaN
3,10922788,06AUG2014:00:00:00,21:54:49,86.00,RESTORAN,999999,0,AV,TURK_LIRASI,NaN,NaN
4,10335987,12APR2015:00:00:00,19:10:09,30.00,RESTORAN,999999,0,AV,TURK_LIRASI,NaN,NaN


In [12]:
df.rename(columns={'MUSTERI_ID_MASK' : 'customer_id',
'ISLEM_TARIHI':'Trans_date',
'ISLEM_SAAT':'Trans_time',
'ISLEM_TUTARI':'Trans_tot_amount',
'MCC':'Spending_cat',
'UYEISYERI_ID_MASK':'merchant_id',
'ONLINE_ISLEM':'Online_payment',
'HARCAMA_TIPI':'Trans_type',
'DOVIZ_FMT':'Trans_currency',
'X':'Merch_loc_X',
'Y':'Merch_loc_Y'},inplace=True)

In [15]:
df.value_counts(['Spending_cat'])

Spending_cat                          
GIDA                                      2718699
RESTORAN                                  1265400
TEKSTÝL                                    657625
AKARYAKIT                                  646976
TELEKOMÜNÝKASYON                           343764
SAÐLIK                                     319314
DÝÐER                                      300104
HÝZMET SEKTÖRLERÝ                          252031
TEKNOLOJÝ                                  189665
NAKÝT AVANS                                148677
MOBÝLYA, DEKORASYON                        125705
KOZMETÝK                                   124840
SÝGORTA-MAIL ORDER                         103323
SEYAHAT ACENTALARI - TAÞIMACILIK           101652
AYAKKABI                                    97174
YAPI MALZ., HIRDAVAT, NALBURÝYE             75576
OTOMOTÝV                                    71630
SPOR GÝYÝM                                  66044
MUZIK MARKET KIRTASÝYE                      60341
HOTEL      

In [14]:
df.value_counts(['Online_payment'])

Online_payment
0                 8517023
1                  817602
dtype: int64

In [ ]:
custs = df.customer_id.unique()
len(custs)

102893

In [ ]:
#df.dtypes

In [ ]:
from sklearn.preprocessing import LabelEncoder
lenc = LabelEncoder()
df['Spending_enc'] = lenc.fit_transform(df['Spending_cat'])

In [ ]:
a = pd.unique(df[['Spending_enc','Spending_cat']].values.ravel())
b = a[::2]
c = a[1::2]
dic = {'encode_val':b,'base_val':c}
d = pd.DataFrame(dic)
d

,encode_val,base_val
0,23,RESTORAN
1,35,NaN
2,25,SEYAHAT ACENTALARI - TAÞIMACILIK
3,11,HAVAYOLLARI
4,5,"DENÝZ TAÞITLARI KÝRALAMA, SATIÞ, BAKIM"
5,24,SAÐLIK
6,12,HOTEL
7,7,DÝÐER
8,10,GIDA
9,1,ALIÞVERÝÞ MERKEZLERÝ


In [1]:
df.value_counts()

NameError: name 'df' is not defined

In [10]:
df=df.drop(['Spending_cat'],axis=1)

In [11]:
from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder()

categorical_cols = ['Trans_type','Trans_currency']
labeled_df = df[categorical_cols]
enc = OneHotEncoder()
enc.fit(labeled_df)

onehotlabels = enc.transform(labeled_df).toarray()

new_columns=list()
for col, values in zip(labeled_df.columns, enc.categories_):
    new_columns.extend([col + '_' + str(value) for value in values])

df_out1= pd.concat([df, pd.DataFrame(onehotlabels, columns=new_columns)], axis='columns')
df_out1 = df_out1.drop(categorical_cols,axis=1)

In [12]:
for col in df_out1.columns:
    print(col)

customer_id
Trans_date
Trans_time
Trans_tot_amount
merchant_id
Online_payment
Merch_loc_X
Merch_loc_Y
Spending_enc
Trans_type_AV
Trans_type_nan
Trans_currency_AMERIKAN_DOLARI
Trans_currency_EURO
Trans_currency_TURK_LIRASI


In [14]:
csv_file = "C:/Users/rodri/Desktop/NCFS2/ML/project/data/SU_MUSTERI_KITLE_ORNEKLEM_100K.txt"
df2 = pd.read_csv(csv_file,sep=',',encoding='latin-1')


In [17]:
df2.rename(columns={'MUSTERI_ID_MASK' : 'customer_id',
'MBB_SEGMENT_ACK':'customer_segment',
'SUBE_KODU':'cust_main_branch',
'X_SUBE':'branch_loc_X',
'Y_SUBE':'branch_loc_Y',
'X_EV':'cust_home_loc_X',
'Y_EV':'cust_home_loc_Y',
'X_IS':'cust_work_loc_X',
'Y_IS':'cust_work_loc_Y',
'CINSIYETI':'cust_gender',
'MEDENI_DRM_ACK':'cust_marital',
'EGITIM_DRM_ACK':'cust_education',
'IS_TURU_ACK':'cust_job_status',
'GELIR':'cust_income_bracket',
'YAS':'cust_age',
'BANKA_YASI':'cust_age_wbank',
'KK_RISK_KODU_201407':'missed_payment_jul',
'KK_RISK_KODU_201408':'missed_payment_aug',
'KK_RISK_KODU_201409':'missed_payment_sep',
'KK_RISK_KODU_201410':'missed_payment_oct',
'KK_RISK_KODU_201411':'missed_payment_nov',
'KK_RISK_KODU_201412':'missed_payment_dec',
'KK_RISK_KODU_201501':'missed_payment_jan',
'KK_RISK_KODU_201502':'missed_payment_feb',
'KK_RISK_KODU_201503':'missed_payment_mar',
'KK_RISK_KODU_201504':'missed_payment_apr',
'KK_RISK_KODU_201505':'missed_payment_may',
'KK_RISK_KODU_201506':'missed_payment_jun'
},inplace=True)
df2

,customer_id,customer_segment,cust_main_branch,branch_loc_X,branch_loc_Y,cust_home_loc_X,cust_home_loc_Y,cust_work_loc_X,cust_work_loc_Y,cust_gender,...,missed_payment_sep,missed_payment_oct,missed_payment_nov,missed_payment_dec,missed_payment_jan,missed_payment_feb,missed_payment_mar,missed_payment_apr,missed_payment_may,missed_payment_jun
0,25139695,BÝREYSEL,2,36.991388,35.325423,36.971366,35.362328,41.029810,29.152450,E,...,2)GECIKME 1-15 GUN,1)RISKSIZ,2)GECIKME 1-15 GUN,2)GECIKME 1-15 GUN,1)RISKSIZ,2)GECIKME 1-15 GUN,1)RISKSIZ,2)GECIKME 1-15 GUN,4)GECIKME 30-59 GUN,1)RISKSIZ
1,21213035,BÝREYSEL,2,36.991388,35.325423,37.010670,35.369540,41.070440,28.989410,E,...,1)RISKSIZ,1)RISKSIZ,1)RISKSIZ,2)GECIKME 1-15 GUN,1)RISKSIZ,2)GECIKME 1-15 GUN,2)GECIKME 1-15 GUN,1)RISKSIZ,2)GECIKME 1-15 GUN,2)GECIKME 1-15 GUN
2,21242221,BÝREYSEL,2,36.991388,35.325423,40.920454,29.187872,36.995320,35.274170,E,...,4)GECIKME 30-59 GUN,1)RISKSIZ,2)GECIKME 1-15 GUN,4)GECIKME 30-59 GUN,5)GECIKME 60+ GUN,5)GECIKME 60+ GUN,6)TAKIP,6)TAKIP,6)TAKIP,6)TAKIP
3,23187800,BÝREYSEL,2,36.991388,35.325423,41.100502,28.892116,41.090251,28.896666,E,...,4)GECIKME 30-59 GUN,1)RISKSIZ,1)RISKSIZ,1)RISKSIZ,1)RISKSIZ,1)RISKSIZ,1)RISKSIZ,1)RISKSIZ,1)RISKSIZ,1)RISKSIZ
4,16400475,BÝREBÝR,2,36.991388,35.325423,41.008290,29.083431,40.984750,29.166130,E,...,1)RISKSIZ,1)RISKSIZ,1)RISKSIZ,2)GECIKME 1-15 GUN,4)GECIKME 30-59 GUN,5)GECIKME 60+ GUN,5)GECIKME 60+ GUN,6)TAKIP,6)TAKIP,6)TAKIP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103204,28039819,BÝREYSEL,1442,38.474103,27.083277,40.913335,29.261285,41.119680,29.051570,E,...,.,.,.,.,.,.,.,.,1)RISKSIZ,1)RISKSIZ
103205,24777429,BÝREYSEL,1442,38.474103,27.083277,41.016540,29.117130,41.023570,29.038930,E,...,.,.,.,.,.,.,.,.,1)RISKSIZ,1)RISKSIZ
103206,24610106,BÝREYSEL,1443,40.767618,29.966363,41.041529,28.998523,40.773060,29.962990,E,...,1)RISKSIZ,2)GECIKME 1-15 GUN,2)GECIKME 1-15 GUN,1)RISKSIZ,1)RISKSIZ,1)RISKSIZ,1)RISKSIZ,1)RISKSIZ,1)RISKSIZ,1)RISKSIZ
103207,15871198,BÝREYSEL,1443,40.767618,29.966363,41.098674,28.860260,40.773400,29.961979,E,...,.,.,.,.,.,.,.,.,1)RISKSIZ,1)RISKSIZ


In [16]:
custs = df2.customer_id.unique()
len(custs)

103209

In [7]:
for col in df2.columns:
    print(col)

customer_id
customer_segment
cust_main_branch
branch_loc_X
branch_loc_Y
cust_home_loc_X
cust_home_loc_Y
cust_work_loc_X
cust_work_loc_Y
cust_gender
cust_marital
cust_education
cust_job_status
cust_income_bracket
cust_age
cust_age_wbank
missed_payment_jul
missed_payment_aug
missed_payment_sep
missed_payment_oct
missed_payment_nov
missed_payment_dec
missed_payment_jan
missed_payment_feb
missed_payment_mar
missed_payment_apr
missed_payment_may
missed_payment_jun


In [15]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()
categorical_feature_mask = df2.dtypes==object
categorical_cols = df2.columns[categorical_feature_mask].tolist()
labeled_df = df2[categorical_cols]
enc = OneHotEncoder()
enc.fit(labeled_df)

onehotlabels = enc.transform(labeled_df).toarray()

new_columns=list()
for col, values in zip(labeled_df.columns, enc.categories_):
    new_columns.extend([col + '_' + str(value) for value in values])

df_out2= pd.concat([df2, pd.DataFrame(onehotlabels, columns=new_columns)], axis='columns')
df_out2 = df_out2.drop(categorical_cols,axis=1)

In [ ]:
for col in df_out2.columns:
    print(col)

In [16]:
df_out2.rename(columns={
'customer_segment_BÝREBÝR':'p2p',
'customer_segment_BÝREYSEL':'individual',
'customer_segment_EXI26':'EXI',
'customer_segment_MÝKRO':'micro',
'customer_segment_ÖZEL BANKACILIK MÜÞTERÝLERÝ':'special',
'missed_payment_jul_1)RISKSIZ':'jul_none',
'missed_payment_jul_2)GECIKME 1-15 GUN':'jul_little_late',
'missed_payment_jul_3)GECIKME 16-29 GUN':'jul_late',
'missed_payment_jul_4)GECIKME 30-59 GUN':'jul_very_late',
'missed_payment_jul_5)GECIKME 60+ GUN':'jul_extremely_late',
'missed_payment_jul_6)TAKIP':'jul_followup',
'missed_payment_aug_1)RISKSIZ':'aug_none',
'missed_payment_aug_2)GECIKME 1-15 GUN':'aug_little_late',
'missed_payment_aug_3)GECIKME 16-29 GUN':'aug_late',
'missed_payment_aug_4)GECIKME 30-59 GUN':'aug_very_late',
'missed_payment_aug_5)GECIKME 60+ GUN':'aug_extremely_late',
'missed_payment_aug_6)TAKIP':'aug_followup',
'missed_payment_sep_1)RISKSIZ':'sep_none',
'missed_payment_sep_2)GECIKME 1-15 GUN':'sep_little_late',
'missed_payment_sep_3)GECIKME 16-29 GUN':'sep_late',
'missed_payment_sep_4)GECIKME 30-59 GUN':'sep_very_late',
'missed_payment_sep_5)GECIKME 60+ GUN':'sep_extremely_late',
'missed_payment_sep_6)TAKIP':'sep_followup',
'missed_payment_oct_1)RISKSIZ':'oct_none',
'missed_payment_oct_2)GECIKME 1-15 GUN':'oct_little_late',
'missed_payment_oct_3)GECIKME 16-29 GUN':'oct_late',
'missed_payment_oct_4)GECIKME 30-59 GUN':'oct_very_late',
'missed_payment_oct_5)GECIKME 60+ GUN':'oct_extremely_late',
'missed_payment_oct_6)TAKIP':'oct_followup',
'missed_payment_nov_1)RISKSIZ':'nov_none',
'missed_payment_nov_2)GECIKME 1-15 GUN':'nov_little_late',
'missed_payment_nov_3)GECIKME 16-29 GUN':'nov_late',
'missed_payment_nov_4)GECIKME 30-59 GUN':'nov_very_late',
'missed_payment_nov_5)GECIKME 60+ GUN':'nov_extremely_late',
'missed_payment_nov_6)TAKIP':'nov_followup',
'missed_payment_dec_1)RISKSIZ':'dec_none',
'missed_payment_dec_2)GECIKME 1-15 GUN':'dec_little_late',
'missed_payment_dec_3)GECIKME 16-29 GUN':'dec_late',
'missed_payment_dec_4)GECIKME 30-59 GUN':'dec_very_late',
'missed_payment_dec_5)GECIKME 60+ GUN':'dec_extremely_late',
'missed_payment_dec_6)TAKIP':'dec_followup',
'missed_payment_jan_1)RISKSIZ':'jan_none',
'missed_payment_jan_2)GECIKME 1-15 GUN':'jan_little_late',
'missed_payment_jan_3)GECIKME 16-29 GUN':'jan_late',
'missed_payment_jan_4)GECIKME 30-59 GUN':'jan_very_late',
'missed_payment_jan_5)GECIKME 60+ GUN':'jan_extremely_late',
'missed_payment_jan_6)TAKIP':'jan_followup',
'missed_payment_feb_1)RISKSIZ':'feb_none',
'missed_payment_feb_2)GECIKME 1-15 GUN':'feb_little_late',
'missed_payment_feb_3)GECIKME 16-29 GUN':'feb_late',
'missed_payment_feb_4)GECIKME 30-59 GUN':'feb_very_late',
'missed_payment_feb_5)GECIKME 60+ GUN':'feb_extremely_late',
'missed_payment_feb_6)TAKIP':'feb_followup',
'missed_payment_mar_1)RISKSIZ':'mar_none',
'missed_payment_mar_2)GECIKME 1-15 GUN':'mar_little_late',
'missed_payment_mar_3)GECIKME 16-29 GUN':'mar_late',
'missed_payment_mar_4)GECIKME 30-59 GUN':'mar_very_late',
'missed_payment_mar_5)GECIKME 60+ GUN':'mar_extremely_late',
'missed_payment_mar_6)TAKIP':'mar_followup',
'missed_payment_apr_1)RISKSIZ':'apr_none',
'missed_payment_apr_2)GECIKME 1-15 GUN':'apr_little_late',
'missed_payment_apr_3)GECIKME 16-29 GUN':'apr_late',
'missed_payment_apr_4)GECIKME 30-59 GUN':'apr_very_late',
'missed_payment_apr_5)GECIKME 60+ GUN':'apr_extremely_late',
'missed_payment_apr_6)TAKIP':'apr_followup',
'missed_payment_may_1)RISKSIZ':'may_none',
'missed_payment_may_2)GECIKME 1-15 GUN':'may_little_late',
'missed_payment_may_3)GECIKME 16-29 GUN':'may_late',
'missed_payment_may_4)GECIKME 30-59 GUN':'may_very_late',
'missed_payment_may_5)GECIKME 60+ GUN':'may_extremely_late',
'missed_payment_may_6)TAKIP':'may_followup',
'missed_payment_jun_1)RISKSIZ':'jun_none',
'missed_payment_jun_2)GECIKME 1-15 GUN':'jun_little_late',
'missed_payment_jun_3)GECIKME 16-29 GUN':'jun_late',
'missed_payment_jun_4)GECIKME 30-59 GUN':'jun_very_late',
'missed_payment_jun_5)GECIKME 60+ GUN':'jun_extremely_late',
'missed_payment_jun_6)TAKIP':'jun_followup',
},inplace=True)

In [ ]:
df_out2.head()

In [6]:
df3 = df.merge(df2,how='inner',on='customer_id')

In [ ]:
from sklearn.preprocessing import LabelEncoder
lenc = LabelEncoder()
df3['Spending_enc'] = lenc.fit_transform(df3['Spending_cat'])
df3=df3.drop(['Spending_cat'],axis=1)

In [9]:
from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder()

categorical_cols = ['Trans_type','Trans_currency','cust_gender', 'cust_marital', 'cust_education', 'cust_job_status']
labeled_df = df3[categorical_cols]
enc = OneHotEncoder()
enc.fit(labeled_df)

onehotlabels = enc.transform(labeled_df).toarray()

new_columns=list()
for col, values in zip(labeled_df.columns, enc.categories_):
    new_columns.extend([col + '_' + str(value) for value in values])

df_out3= pd.concat([df3, pd.DataFrame(onehotlabels, columns=new_columns)], axis='columns')
df_out3 = df_out3.drop(categorical_cols,axis=1)

MemoryError: Unable to allocate 2.43 GiB for an array with shape (35, 9334625) and data type float64

In [ ]:
df_out3.rename(columns={
'customer_segment_BÝREBÝR':'p2p',
'customer_segment_BÝREYSEL':'individual',
'customer_segment_EXI26':'EXI',
'customer_segment_MÝKRO':'micro',
'customer_segment_ÖZEL BANKACILIK MÜÞTERÝLERÝ':'special',
'missed_payment_jul_1)RISKSIZ':'jul_none',
'missed_payment_jul_2)GECIKME 1-15 GUN':'jul_little_late',
'missed_payment_jul_3)GECIKME 16-29 GUN':'jul_late',
'missed_payment_jul_4)GECIKME 30-59 GUN':'jul_very_late',
'missed_payment_jul_5)GECIKME 60+ GUN':'jul_extremely_late',
'missed_payment_jul_6)TAKIP':'jul_followup',
'missed_payment_aug_1)RISKSIZ':'aug_none',
'missed_payment_aug_2)GECIKME 1-15 GUN':'aug_little_late',
'missed_payment_aug_3)GECIKME 16-29 GUN':'aug_late',
'missed_payment_aug_4)GECIKME 30-59 GUN':'aug_very_late',
'missed_payment_aug_5)GECIKME 60+ GUN':'aug_extremely_late',
'missed_payment_aug_6)TAKIP':'aug_followup',
'missed_payment_sep_1)RISKSIZ':'sep_none',
'missed_payment_sep_2)GECIKME 1-15 GUN':'sep_little_late',
'missed_payment_sep_3)GECIKME 16-29 GUN':'sep_late',
'missed_payment_sep_4)GECIKME 30-59 GUN':'sep_very_late',
'missed_payment_sep_5)GECIKME 60+ GUN':'sep_extremely_late',
'missed_payment_sep_6)TAKIP':'sep_followup',
'missed_payment_oct_1)RISKSIZ':'oct_none',
'missed_payment_oct_2)GECIKME 1-15 GUN':'oct_little_late',
'missed_payment_oct_3)GECIKME 16-29 GUN':'oct_late',
'missed_payment_oct_4)GECIKME 30-59 GUN':'oct_very_late',
'missed_payment_oct_5)GECIKME 60+ GUN':'oct_extremely_late',
'missed_payment_oct_6)TAKIP':'oct_followup',
'missed_payment_nov_1)RISKSIZ':'nov_none',
'missed_payment_nov_2)GECIKME 1-15 GUN':'nov_little_late',
'missed_payment_nov_3)GECIKME 16-29 GUN':'nov_late',
'missed_payment_nov_4)GECIKME 30-59 GUN':'nov_very_late',
'missed_payment_nov_5)GECIKME 60+ GUN':'nov_extremely_late',
'missed_payment_nov_6)TAKIP':'nov_followup',
'missed_payment_dec_1)RISKSIZ':'dec_none',
'missed_payment_dec_2)GECIKME 1-15 GUN':'dec_little_late',
'missed_payment_dec_3)GECIKME 16-29 GUN':'dec_late',
'missed_payment_dec_4)GECIKME 30-59 GUN':'dec_very_late',
'missed_payment_dec_5)GECIKME 60+ GUN':'dec_extremely_late',
'missed_payment_dec_6)TAKIP':'dec_followup',
'missed_payment_jan_1)RISKSIZ':'jan_none',
'missed_payment_jan_2)GECIKME 1-15 GUN':'jan_little_late',
'missed_payment_jan_3)GECIKME 16-29 GUN':'jan_late',
'missed_payment_jan_4)GECIKME 30-59 GUN':'jan_very_late',
'missed_payment_jan_5)GECIKME 60+ GUN':'jan_extremely_late',
'missed_payment_jan_6)TAKIP':'jan_followup',
'missed_payment_feb_1)RISKSIZ':'feb_none',
'missed_payment_feb_2)GECIKME 1-15 GUN':'feb_little_late',
'missed_payment_feb_3)GECIKME 16-29 GUN':'feb_late',
'missed_payment_feb_4)GECIKME 30-59 GUN':'feb_very_late',
'missed_payment_feb_5)GECIKME 60+ GUN':'feb_extremely_late',
'missed_payment_feb_6)TAKIP':'feb_followup',
'missed_payment_mar_1)RISKSIZ':'mar_none',
'missed_payment_mar_2)GECIKME 1-15 GUN':'mar_little_late',
'missed_payment_mar_3)GECIKME 16-29 GUN':'mar_late',
'missed_payment_mar_4)GECIKME 30-59 GUN':'mar_very_late',
'missed_payment_mar_5)GECIKME 60+ GUN':'mar_extremely_late',
'missed_payment_mar_6)TAKIP':'mar_followup',
'missed_payment_apr_1)RISKSIZ':'apr_none',
'missed_payment_apr_2)GECIKME 1-15 GUN':'apr_little_late',
'missed_payment_apr_3)GECIKME 16-29 GUN':'apr_late',
'missed_payment_apr_4)GECIKME 30-59 GUN':'apr_very_late',
'missed_payment_apr_5)GECIKME 60+ GUN':'apr_extremely_late',
'missed_payment_apr_6)TAKIP':'apr_followup',
'missed_payment_may_1)RISKSIZ':'may_none',
'missed_payment_may_2)GECIKME 1-15 GUN':'may_little_late',
'missed_payment_may_3)GECIKME 16-29 GUN':'may_late',
'missed_payment_may_4)GECIKME 30-59 GUN':'may_very_late',
'missed_payment_may_5)GECIKME 60+ GUN':'may_extremely_late',
'missed_payment_may_6)TAKIP':'may_followup',
'missed_payment_jun_1)RISKSIZ':'jun_none',
'missed_payment_jun_2)GECIKME 1-15 GUN':'jun_little_late',
'missed_payment_jun_3)GECIKME 16-29 GUN':'jun_late',
'missed_payment_jun_4)GECIKME 30-59 GUN':'jun_very_late',
'missed_payment_jun_5)GECIKME 60+ GUN':'jun_extremely_late',
'missed_payment_jun_6)TAKIP':'jun_followup',
},inplace=True)

In [17]:
df_out1.to_csv(r"C:/Users/rodri/Desktop/NCFS2/ML/project/data/transactions.csv",index=False)

In [18]:
df_out2.to_csv(r"C:/Users/rodri/Desktop/NCFS2/ML/project/data/demographics.csv",index=False)